In [124]:
%reload_ext autoreload
%autoreload 2

import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from scripts import test_config, setup
from src import pipeline
from src.storage import retrieve

# set up data stores
config_data = test_config.get_config_data()
setup.initialize_data_stores(config_data)

# backfill data
scenerio_data = config_data["test"]["scenerio"][0]
pipeline.backfill_stm(config_data, scenerio_data)

# make bots available
bot_datas = config_data["test"]["bot_datas"]

In [ ]:
def ask_bot_questions(bot_user_id, questions):
    bot_data = bot_datas[bot_user_id]
    for question in questions:
        pipeline.answer_question(config_data, scenerio_data, bot_data, question)

ask_bot_questions(bot_datas["alice"], config_data["test"]["questions"])

In [ ]:
def get_bot_response(bot_user_id):
    bot_data = bot_datas[bot_user_id]
    message = retrieve.get_latest_event(config_data, scenerio_data)
    response = pipeline.chat(config_data, scenerio_data, bot_data, message)
    pipeline.update_stm(config_data, scenerio_data, bot_data, response)

get_bot_response(bot_datas["alice"])

In [ ]:
def update_memory():
    message = retrieve.get_latest_event(config_data, scenerio_data)
    pipeline.update_mtm(config_data, scenerio_data, message)
    pipeline.update_ltm(config_data, scenerio_data, message)

update_memory()

In [ ]:
# print all the memory stores?
print(config_data["summarize"]["mtm_template"])